# Korean Movie Recommender system

목적: 데이터에서 user의 각 영화에 대한 평점을 바탕으로 출연한 주연 배우에 대한 평점을 계산하고 user 정보와 주연 배우에 대한 평점을 이용한 협업필터링을 통해 user에게 영화를 추천하는 알고리즘을 구현해본다

## 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import joblib
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## 데이터 불러오기

본 실습에서 필요한 정보는 주연배우와 영화, 평점이지만, 데이터셋 내의 전체 data scheme을 확인하기 위해 전체 테이블을 불러와 시각화

In [2]:
from kmr_dataset import load_rates
from kmr_dataset import get_paths

paths = get_paths(size='5m')

/Users/jihong/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
castings, countries, genres, movies, peoples, rates = [pd.read_csv(path, delimiter = delim) for path, delim in zip(paths, [None] * 3 +['\t'] * 2 + [None])]

In [4]:
def check_scheme(df_names):
    df_list = [globals()[name] for name in df_names]
    whole_columns = []
    for df in df_list:
        whole_columns.extend(df.columns.tolist())
    whole_columns = list(set(whole_columns))
    tbl_rel = pd.DataFrame(index = whole_columns, columns = df_names)
    for idx, col in enumerate(tbl_rel.columns):
        for row in tbl_rel.index:
            tbl_rel.loc[row, col] = row in df_list[idx]
    return tbl_rel.astype(int).style.background_gradient(cmap = "Blues")

In [5]:
check_scheme(["castings", "countries", "genres", "movies", "peoples", "rates"])

,castings,countries,genres,movies,peoples,rates
title,0,0,0,1,0,0
year,0,0,0,1,0,0
korean,0,0,0,0,1,0
original,0,0,0,0,1,0
movie,1,1,1,1,0,1
genre,0,0,1,0,0,0
order,1,0,0,0,0,0
leading,1,0,0,0,0,0
title_eng,0,0,0,1,0,0
time,0,0,0,0,0,1


In [6]:
castings.shape, countries.shape, genres.shape, movies.shape, peoples.shape, rates.shape

((819743, 4), (193783, 2), (197624, 2), (180982, 5), (263438, 3), (5031052, 4))

### 테이블 병합 전략
* `peoples`를 제외한 `castings, countries,	genres,	movies,	peoples`에는 `movie` 컬럼을 가지고 있으므로, `movies`테이블 기준으로 테이블을 병합 후 `rates`에 `movie`를 기준으로 병합

#### `castings`
* 한 영화에 출연하는 여러명의 배우 확인

In [7]:
castings = castings.set_index('movie')

In [8]:
castings

,people,order,leading
movie,,,
10001,4374,1,1
10001,178,2,1
10001,3241,3,1
10001,47952,4,1
10001,47953,5,0
...,...,...,...
190978,433829,4,1
190991,2013,1,1
190991,387343,2,1


#### `countries`
* 각 영화별로 국가가 2개 이상인 경우 존재 -> 한 행에 병기하는 방식으로 전처리

In [9]:
print(countries["movie"].nunique())
countries.head()

171166


,movie,country
0,10001,이탈리아
1,10001,프랑스
2,10002,미국
3,10003,미국
4,10004,미국


In [10]:
countries =  countries.groupby("movie").agg({"country" : lambda x : '/'.join(x)})
print(countries.index.nunique())
countries.head()

171166


,country
movie,
10001,이탈리아/프랑스
10002,미국
10003,미국
10004,미국
10005,미국


#### `genres`
* 각 영화별로 장르가 복수의 행으로 존재 -> 한 행에 병기하는 방식으로 전처리

In [11]:
print(genres["movie"].nunique())
genres.head()

129090


,movie,genre
0,10001,드라마
1,10001,멜로/로맨스
2,10002,SF
3,10002,코미디
4,10003,SF


In [12]:
genres =  genres.groupby("movie").agg({"genre" : lambda x : '/'.join(x)})
print(genres.index.nunique())
genres.head()

129090


,genre
movie,
10001,드라마/멜로/로맨스
10002,SF/코미디
10003,SF/코미디
10004,서부/SF/판타지/코미디
10005,판타지/모험/SF/액션


#### `movies`
* 영화가 중복되는 행은 없음

In [13]:
movies.shape, movies["movie"].nunique()

((180982, 5), 180982)

In [14]:
movies = movies.set_index("movie")
movies.head()

,title,title_eng,year,grade
movie,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",1987,12세 관람가
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",1990,12세 관람가
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990,전체 관람가
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1978,PG


#### `peoples`
* `casting` 테이블의 배우들의 번호에 대한 dictionary로 활용

In [15]:
peoples

,people,korean,original
0,1,알리야,Aaliyah
1,2,압바스 키아로스타미,Abbas Kiarostami
2,3,아벨 페라라,Abel Ferrara
3,4,아담 아킨,Adam Arkin
4,5,아담 볼드윈,Adam Baldwin
...,...,...,...
263433,435622,소우진,NaN
263434,435623,임건우,NaN
263435,435624,조윤영,NaN
263436,435625,노하은,NaN


* 배우 id와 이름을 매핑하기 위한 dictionary 생성

In [16]:
from collections import defaultdict
people_dict = defaultdict(dict)

In [17]:
for p, k, o in zip(peoples["people"].tolist(), peoples["korean"].tolist(), peoples["original"].tolist()):
    people_dict[p] = {"korean": k, "original" : o}

In [18]:
people_dict

defaultdict(dict,
            {1: {'korean': '알리야', 'original': 'Aaliyah'},
             2: {'korean': '압바스 키아로스타미', 'original': 'Abbas Kiarostami'},
             3: {'korean': '아벨 페라라', 'original': 'Abel Ferrara'},
             4: {'korean': '아담 아킨', 'original': 'Adam Arkin'},
             5: {'korean': '아담 볼드윈', 'original': 'Adam Baldwin'},
             6: {'korean': '아담 샌들러', 'original': 'Adam Sandler'},
             7: {'korean': '아데웰 아킨누오예 아바제',
              'original': 'Adewale Akinnuoye-Agbaje'},
             8: {'korean': '애드리안 라인', 'original': 'Adrian Lyne'},
             9: {'korean': '에이단 퀸', 'original': 'Aidan Quinn'},
             10: {'korean': '아키 카우리스마키', 'original': 'Aki Kaurismaki'},
             11: {'korean': '에모토 아키라', 'original': 'Emoto Akira'},
             12: {'korean': '테라오 아키라', 'original': 'Akira Terao'},
             13: {'korean': '구로사와 아키라', 'original': 'Akira Kurosawa'},
             14: {'korean': '알 애덤슨', 'original': 'Al Adamson'},
             15: {'

* casting 테이블과 movie 테이블을 영화 id index를 기준으로 합치기

In [19]:
movie_infos = movies.join(castings)
movie_infos.head()

,title,title_eng,year,grade,people,order,leading
movie,,,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,4374.0,1.0,1.0
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,178.0,2.0,1.0
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,3241.0,3.0,1.0
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,47952.0,4.0,1.0
10001,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,47953.0,5.0,0.0


In [20]:
movie_infos.nunique()

title        150620
title_eng    156554
year            101
grade            11
people       187975
order           245
leading           2
dtype: int64

* 각 배우가 출연한 영화를 리스트로 반환하는 dictionary 생성

In [21]:
movie_by_cast = castings[["people"]].reset_index().groupby("people").agg({"movie" : lambda x : list(x)})
movie_by_cast

,movie
people,
1,"[29152, 34388]"
2,"[27220, 33645, 37694, 49263, 84185]"
3,"[28870, 33044, 65104, 65472, 69797, 71832, 102..."
4,"[14606, 16421, 17479, 17620, 24407, 25822, 270..."
5,"[10062, 13612, 15164, 15188, 16598, 17347, 174..."
...,...
435622,[190628]
435623,[190628]
435624,[190628]


In [22]:
movie_by_cast_dict = defaultdict(list)

In [23]:
for c, m in zip(movie_by_cast.index, movie_by_cast["movie"].tolist()):
    movie_by_cast_dict[c] = m

In [24]:
movie_by_cast_dict[1]

[29152, 34388]

#### `rates`

In [25]:
rates.describe()

,user,movie,rate,time
count,5.031052e+06,5.031052e+06,5.031052e+06,5.031052e+06
mean,2.241073e+04,8.190041e+04,7.471898e+00,1.357414e+09
std,2.437763e+04,4.551494e+04,2.960652e+00,1.349618e+08
min,0.000000e+00,1.000100e+04,0.000000e+00,1.069340e+09
25%,2.109000e+03,4.488500e+04,6.000000e+00,1.241968e+09
50%,1.221250e+04,7.162800e+04,8.000000e+00,1.367593e+09
75%,3.743400e+04,1.178790e+05,1.000000e+01,1.475939e+09
max,8.645600e+04,1.912370e+05,1.000000e+01,1.578496e+09


In [26]:
rates = rates.drop_duplicates()
rates.isnull().sum()

user     0
movie    0
rate     0
time     0
dtype: int64

In [27]:
rates["user"].nunique(), rates["movie"].nunique()

(86457, 48891)

* 한 영화에 대한 user의 평점이 중복되는 경우를 고려하여, 평균 평점을 계산

In [28]:
rates = rates.groupby(["user", "movie"])[["rate"]].mean().reset_index("user")
rates

,user,rate
movie,,
10002,0,10.0
10003,0,9.0
10004,0,9.0
10005,0,5.0
10006,0,3.0
...,...,...
140718,86456,8.0
141104,86456,8.0
144927,86456,8.0


* 영화 정보와 평점을 결합

In [29]:
df = rates.join(movie_infos)
df.head()

,user,rate,title,title_eng,year,grade,people,order,leading
movie,,,,,,,,,
10001,2,7.0,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,4374.0,1.0,1.0
10001,2,7.0,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,178.0,2.0,1.0
10001,2,7.0,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,3241.0,3.0,1.0
10001,2,7.0,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,47952.0,4.0,1.0
10001,2,7.0,시네마 천국,"Cinema Paradiso , 1988",1990,전체 관람가,47953.0,5.0,0.0


* 제목에 결측치가 있으면 추천 자체가 안되므로, 제목에 결측치가 있는 데이터 제외

In [30]:
df.isnull().sum()

user               0
rate               0
title          25855
title_eng      29080
year              36
grade        1650420
people         48108
order          48108
leading        48108
dtype: int64

In [31]:
df = df[df["title"].notnull()].copy()
df.shape

(67599767, 9)

* 과제 목적에 부합하지 않는 테이블은 삭제

In [32]:
del castings, countries, genres, movies, peoples

### `people` 전처리 전략

* `leading`을 참고하여 주연배우들만 추천에 활용
* `order`는 무시

In [33]:
df = df.loc[df["leading"] == 1, ["user", "rate", "title", "people"]].copy()
df.shape

(18315785, 4)

In [34]:
df["people"] = df["people"].astype(int)

* `surprise` 데이터셋으로 변환하기 위해 `['user', 'people', 'rate']` 순의 데이터로 변환

In [35]:
df = df[["user", "people", "rate"]].copy()

In [36]:
df

,user,people,rate
movie,,,
10001,2,4374,7.0
10001,2,178,7.0
10001,2,3241,7.0
10001,2,47952,7.0
10001,3,4374,10.0
...,...,...,...
190855,70562,67893,10.0
190978,75681,433832,9.0
190978,75681,433831,9.0


In [37]:
df.shape

(18315785, 3)

* 각 user의 주연배우에 대한 평점이 영화별로 다를 수 있으므로, 평균하여 사용

In [38]:
df = df.groupby(["user", "people"]).mean("rate").reset_index()

* 평점의 range는 (0, 10)

In [39]:
r_min = df["rate"].min()
r_max = df["rate"].max()
r_min, r_max

(0.0, 10.0)

* surprise Dataset으로 변환

In [40]:
reader = Reader(rating_scale = (r_min, r_max))

In [41]:
df_train, df_test = train_test_split(df, test_size = 0.2,  random_state = 42)

In [42]:
df_train.shape, df_test.shape

((11992742, 3), (2998186, 3))

In [43]:
train_data = Dataset.load_from_df(df_train, reader)
test_data = Dataset.load_from_df(df_test, reader)

In [44]:
X_train = train_data.build_full_trainset()
X_test = test_data.build_full_trainset().build_testset()

In [45]:
X_train.n_users, X_train.n_items, X_train.n_ratings

(86457, 44505, 11992742)

In [46]:
type(X_test), len(X_test)

(list, 2998186)

* 특이값 분해 모델로 학습

In [47]:
svd = SVD()

In [49]:
svd.fit(X_train)

* 학습된 모델을 저장

In [51]:
model_path = "people_svd.joblib.gzip"
joblib.dump(svd, model_path, compress = "gzip")

* 학습한 모델 로드 후 예측해보기

In [52]:
model = joblib.load("../best_svd.joblib.gzip")

In [53]:
predictions = model.test(X_test)

In [54]:
predictions

[Prediction(uid=2366, iid=122174, r_ui=1.0, est=9.522848105292239, details={'was_impossible': False}),
 Prediction(uid=2366, iid=309351, r_ui=9.0, est=8.101605051731186, details={'was_impossible': False}),
 Prediction(uid=2366, iid=5686, r_ui=7.0, est=2.6826698050735605, details={'was_impossible': False}),
 Prediction(uid=2366, iid=920, r_ui=7.0, est=7.206801879071839, details={'was_impossible': False}),
 Prediction(uid=2366, iid=120197, r_ui=7.0, est=3.6570608582276396, details={'was_impossible': False}),
 Prediction(uid=2366, iid=40169, r_ui=1.0, est=5.262322115079651, details={'was_impossible': False}),
 Prediction(uid=2366, iid=552, r_ui=8.0, est=8.041043925120125, details={'was_impossible': False}),
 Prediction(uid=2366, iid=90131, r_ui=7.0, est=10.0, details={'was_impossible': False}),
 Prediction(uid=2366, iid=1054, r_ui=10.0, est=8.513401917288924, details={'was_impossible': False}),
 Prediction(uid=2366, iid=39842, r_ui=5.0, est=3.4281365742830516, details={'was_impossible': F

* Test score (RMSE)

In [55]:
accuracy.rmse(predictions)

RMSE: 2.1736


2.1735994120813245

#### 새로운 사용자가 입력한 영화에 대한 평점을 바탕으로 해당 사용자와 유사한 사용자의 영화배우 평점을 바탕으로 추천해보기

* 평점 입력받을 영화 임의로 선정

In [58]:
chosen_index = np.random.choice(rates.index, 5)
chosen_index

array([ 56232, 126032,  35273, 149747, 118955])

In [74]:
chosen_movie = rates.loc[chosen_index]
chosen_movie

,user,rate
movie,,
56232,0,2.0
56232,1,2.0
56232,2,6.0
56232,4,4.0
56232,5,4.0
...,...,...
118955,86004,10.0
118955,86008,8.0
118955,86222,10.0


* 새로운 사용자의 평점 기록 (여기서는 0~10 임의값)

In [100]:
new_user_rate = np.random.rand(5) * 10
new_user_rate

array([4.11481963, 8.53960639, 4.71182246, 5.20024273, 9.5534141 ])

In [101]:
rcv_rate = pd.DataFrame({"user" : ["new"] * 5, "movie" : chosen_index, "rate" : new_user_rate.tolist()}).pivot_table(index = "user", columns = "movie", values= "rate", aggfunc= "mean", fill_value = 5)

In [102]:
rcv_rate

movie,35273,56232,118955,126032,149747
user,,,,,
new,4.711822,4.11482,9.553414,8.539606,5.200243


In [103]:
cs_matrix = chosen_movie.reset_index().pivot_table(index = "user", columns = "movie", values= "rate", aggfunc= "sum", fill_value = 5)
cs_matrix

movie,35273,56232,118955,126032,149747
user,,,,,
0,5,2.0,6.0,6.0,3.0
1,5,2.0,6.0,6.0,3.0
2,7,6.0,8.0,7.0,8.0
3,9,5.0,7.0,6.0,10.0
4,10,4.0,7.0,9.0,8.0
...,...,...,...,...,...
86348,5,5.0,5.0,8.0,5.0
86349,5,5.0,5.0,5.0,7.0
86355,5,5.0,2.0,5.0,5.0


* 기존 평점 데이터와 새 사용자의 입력 값에 대한 cosine similarity 계산

In [104]:
user_sim = cosine_similarity(cs_matrix, rcv_rate)

In [105]:
df_user_sim = pd.DataFrame(user_sim, index = cs_matrix.index, columns = rcv_rate.index)
df_user_sim

user,new
user,
0,0.979650
1,0.979650
2,0.958493
3,0.900273
4,0.931429
...,...
86348,0.958053
86349,0.922979
86355,0.852406


* similarity가 가장 높은 사용자 추출하여 해당 사용자에 대해 배우 기반 추천 예측

In [108]:
sim_user_id = df_user_sim['new'].nlargest(1).index[0]
sim_user_id

3568

In [109]:
num_movies = len(movie_by_cast_dict.keys())
num_movies

187975

In [110]:
new_test = list(zip([sim_user_id] * num_movies, movie_by_cast_dict.keys(), [None] * num_movies))
new_test

[(3568, 1, None),
 (3568, 2, None),
 (3568, 3, None),
 (3568, 4, None),
 (3568, 5, None),
 (3568, 6, None),
 (3568, 7, None),
 (3568, 8, None),
 (3568, 9, None),
 (3568, 10, None),
 (3568, 11, None),
 (3568, 12, None),
 (3568, 13, None),
 (3568, 14, None),
 (3568, 15, None),
 (3568, 18, None),
 (3568, 21, None),
 (3568, 22, None),
 (3568, 23, None),
 (3568, 24, None),
 (3568, 26, None),
 (3568, 27, None),
 (3568, 28, None),
 (3568, 29, None),
 (3568, 32, None),
 (3568, 33, None),
 (3568, 34, None),
 (3568, 35, None),
 (3568, 37, None),
 (3568, 38, None),
 (3568, 39, None),
 (3568, 40, None),
 (3568, 41, None),
 (3568, 44, None),
 (3568, 45, None),
 (3568, 47, None),
 (3568, 48, None),
 (3568, 49, None),
 (3568, 50, None),
 (3568, 51, None),
 (3568, 52, None),
 (3568, 53, None),
 (3568, 54, None),
 (3568, 55, None),
 (3568, 56, None),
 (3568, 57, None),
 (3568, 58, None),
 (3568, 59, None),
 (3568, 61, None),
 (3568, 62, None),
 (3568, 63, None),
 (3568, 65, None),
 (3568, 66, None),
 (

* 비슷한 user가 각 배우들에게 가지는 평점 예측

In [111]:
new_predictions = model.test(new_test)

In [112]:
df_pred_svd = pd.DataFrame(new_predictions)
df_pred_svd

,uid,iid,r_ui,est,details
0,3568,1,None,7.183136,{'was_impossible': False}
1,3568,2,None,8.216974,{'was_impossible': False}
2,3568,3,None,7.824664,{'was_impossible': False}
3,3568,4,None,7.715656,{'was_impossible': False}
4,3568,5,None,8.131581,{'was_impossible': False}
...,...,...,...,...,...
187970,3568,435622,None,7.717152,{'was_impossible': False}
187971,3568,435623,None,7.717152,{'was_impossible': False}
187972,3568,435624,None,7.717152,{'was_impossible': False}
187973,3568,435625,None,7.717152,{'was_impossible': False}


* 평점이 가장 높은 상위 배우 추출

In [113]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [119]:
top_num = get_top_n(new_predictions)

In [120]:
df_recomm = pd.DataFrame(top_num.items())

In [121]:
df_recomm

,0,1
0,3568,"[(49, 10.0), (80, 10.0), (114, 10.0), (216, 10..."


In [122]:
user_recomm = pd.DataFrame(df_recomm.iloc[0, 1], columns = ["lead_cast_id", "est_rate"])
user_recomm["lead_cast_name"] = user_recomm["lead_cast_id"].map(people_dict).str["korean"]

* 해당 사용자가 좋아할 것으로 여겨지는 배우 상위 10명

In [123]:
user_recomm

,lead_cast_id,est_rate,lead_cast_name
0,49,10.0,알리 라터
1,80,10.0,아나벨라 시오라
2,114,10.0,바바라 허쉬
3,216,10.0,캐리 엘위스
4,262,10.0,크리스토퍼 리브
5,290,10.0,클린트 이스트우드
6,352,10.0,데니스 퀘이드
7,386,10.0,에드워드 펄롱
8,470,10.0,게리 시나이즈
9,520,10.0,기네스 팰트로


* 최상위 배우를 바탕으로 해당 배우 출연작 리스팅

In [128]:
starring_id = user_recomm["lead_cast_id"].map(movie_by_cast_dict)[0]
starring_id

[25587,
 26972,
 27562,
 29223,
 29573,
 32900,
 33107,
 33207,
 36679,
 38902,
 40208,
 52310,
 53751,
 53825,
 63735,
 65719,
 76182,
 82896,
 83741,
 100675,
 109625,
 110333,
 124551,
 128000,
 137397,
 144354,
 151583]

In [125]:
movie_dict = movie_infos["title"].to_dict()
movie_dict

{10001: '시네마 천국',
 10002: '빽 투 더 퓨쳐',
 10003: '빽 투 더 퓨쳐 2',
 10004: '빽 투 더 퓨쳐 3',
 10005: '스타워즈 에피소드 4 - 새로운 희망',
 10006: '스타워즈 에피소드 5 - 제국의 역습',
 10007: '스타워즈 에피소드 6 - 제다이의 귀환',
 10008: '슈퍼맨',
 10009: '슈퍼맨 2',
 10010: '슈퍼맨 3',
 10011: '슈퍼맨 4 - 최강의 적',
 10012: '다이 하드',
 10013: '킹콩',
 10014: '금지된 장난',
 10015: '타워링',
 10016: '나 홀로 집에',
 10017: '셰인',
 10018: '이티',
 10019: '내일을 향해 쏴라',
 10020: '바람과 함께 사라지다',
 10021: '록키',
 10022: '록키 2',
 10023: '록키 3',
 10024: '록키 4',
 10025: '록키 5',
 10026: '리썰 웨폰',
 10027: '리썰 웨폰 2',
 10028: '터미네이터',
 10029: '죠스',
 10030: '죠스 2',
 10031: '죠스 3',
 10032: '죠스 4',
 10033: '레이더스',
 10034: '인디아나 존스',
 10035: '인디아나 존스 - 최후의 성전',
 10036: '코만도',
 10037: '에이리언',
 10038: '에이리언 2',
 10039: '플래툰',
 10040: '구니스',
 10041: '포세이돈 어드벤쳐',
 10042: '콰이강의 다리',
 10043: '나바론 요새',
 10044: '나바론 요새 2',
 10045: '킬링 필드',
 10046: '빠삐용',
 10047: '스팅',
 10048: '죽은 시인의 사회',
 10049: '그렘린',
 10050: '싸이코',
 10051: '싸이코 2',
 10052: '싸이코 3',
 10053: '자이언트',
 10054: '라 밤바',
 10055: '람보',
 1

In [129]:
recomm_result = [movie_dict[idx] for idx in starring_id]
recomm_result

['그들만의 계절',
 '테크노 스와핑',
 '헌티드 힐',
 '데스티네이션',
 '도슨의 청춘일기',
 '금발이 너무해',
 '파이브 건스',
 '제이 앤 사일런트 밥',
 '데스티네이션 2',
 '쓰리 웨이',
 '우리, 사랑일까요?',
 '옵세스',
 '레지던트 이블 4: 끝나지 않은 전쟁 3D',
 '레지던트 이블 3 - 인류의 멸망',
 '메리골드',
 '히어로즈',
 '크레이지',
 '내셔널 람푼: 스톤에이지',
 '레지던트 이블 5 : 최후의 심판 3D',
 '유아 낫 유',
 '더 에셋',
 '레지던트 이블: 파멸의 날',
 '러브식',
 '레전드 시즌1',
 '레전드 시즌2',
 '컨페스',
 '피치']

## 마무리하며

500만 개 정도의 큰 데이터셋을 가지고 작업을 하다보니, 어떤 방식으로 용량을 줄이고, 데이터를 줄일지 고민이 많았다. 최종적으로 다른 팀원들이 작업한 장르기반, 국가기반과 통합한 추천 시스템을 온라인에 배포하고 싶었지만, 학습된 모델의 용량을 더 줄이지 못해서 구현하지 못한 것이 아쉽다. git-LFS를 활용하면, 100mb이상의 데이터를 처리가능하긴 하지만, 이를 활용하지 않고 배포할 수 있는 루트에 대한 고민이 필요할 것 같다. 데이터 분석은 데이터 처리에 대한 rationale이나 모델에 대한 이해 등 중요한 것이 많지만, 커다란 데이터를 효율적으로 다루는 법에 대한 공부도 필수적이라는 생각이 드는 프로젝트였다.